In [1]:
import tensorflow as tf
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Bidirectional, GlobalMaxPooling1D, Dropout
from keras.layers.experimental.preprocessing import TextVectorization
from keras.models import Sequential
import json
import numpy as np

In [2]:
def load_data_from_jsonl(file_path):
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    return data


train_data = load_data_from_jsonl('train.jsonl')
val_data = load_data_from_jsonl('validation.jsonl')
test_data = load_data_from_jsonl('test.jsonl')

train_texts = [entry['text'] for entry in train_data]
train_labels = [entry['label'] for entry in train_data]

val_texts = [entry['text'] for entry in val_data]
val_labels = [entry['label'] for entry in val_data]

test_texts = [entry['text'] for entry in test_data]
test_labels = [entry['label'] for entry in test_data]

train_labels = np.array(train_labels)
val_labels = np.array(val_labels)

max_words = 30000 
max_sequence_length = 100 
embedding_dim = 100 
lstm_units = 64 
dropout_rate = 0.5  
num_classes = 6 

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train_texts)

In [3]:
train_sequences = tokenizer.texts_to_sequences(train_texts)
val_sequences = tokenizer.texts_to_sequences(val_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

# Pad sequences to ensure consistent length
train_sequences = pad_sequences(train_sequences, maxlen=max_sequence_length)
val_sequences = pad_sequences(val_sequences, maxlen=max_sequence_length)
test_sequences = pad_sequences(test_sequences, maxlen=max_sequence_length)


In [4]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(Bidirectional(LSTM(lstm_units, return_sequences=True)))
model.add(GlobalMaxPooling1D())
model.add(Dropout(dropout_rate))
model.add(Dense(128, activation='relu'))
model.add(Dropout(dropout_rate))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          3000000   
                                                                 
 bidirectional (Bidirection  (None, 100, 128)          84480     
 al)                                                             
                                                                 
 global_max_pooling1d (Glob  (None, 128)               0         
 alMaxPooling1D)                                                 
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0

In [5]:
model.fit(train_sequences, train_labels, epochs=10, batch_size=64, validation_data=(val_sequences, val_labels))


Epoch 1/10
250/250 [==============================] - 43s 158ms/step - loss: 1.3753 - accuracy: 0.4526 - val_loss: 0.7160 - val_accuracy: 0.7190
Epoch 2/10
250/250 [==============================] - 40s 160ms/step - loss: 0.5026 - accuracy: 0.8194 - val_loss: 0.2918 - val_accuracy: 0.8935
Epoch 3/10
250/250 [==============================] - 33s 132ms/step - loss: 0.2252 - accuracy: 0.9222 - val_loss: 0.2228 - val_accuracy: 0.9150
Epoch 4/10
250/250 [==============================] - 34s 135ms/step - loss: 0.1487 - accuracy: 0.9508 - val_loss: 0.2609 - val_accuracy: 0.9080
Epoch 5/10
250/250 [==============================] - 35s 138ms/step - loss: 0.1059 - accuracy: 0.9659 - val_loss: 0.2628 - val_accuracy: 0.9225
Epoch 6/10
250/250 [==============================] - 34s 136ms/step - loss: 0.0840 - accuracy: 0.9721 - val_loss: 0.2738 - val_accuracy: 0.9190
Epoch 7/10
250/250 [==============================] - 40s 159ms/step - loss: 0.0708 - accuracy: 0.9779 - val_loss: 0.2627 - val_ac

In [6]:
model.save("emotion_classification_model.h5")

c:\Users\srini\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [7]:
# Save the tokenizer's word index and configuration
tokenizer_word_index = tokenizer.word_index
tokenizer_config = tokenizer.get_config()

with open("tokenizer_word_index.json", "w") as json_file:
    json.dump(tokenizer_word_index, json_file)

with open("tokenizer_config.json", "w") as json_file:
    json.dump(tokenizer_config, json_file)